In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader 
from torchvision import transforms

from sklearn.utils import class_weight, shuffle
from sklearn.metrics import f1_score, fbeta_score, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
train_dir = '/opt/ml/input/data/train'
train_df = pd.read_csv('/opt/ml/input/data/train/train_data_total.csv')
train_df

,Unnamed: 0,id,gender,img_path,age,age_group,mask,class
0,0,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,mask,4
1,1,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,mask,4
2,2,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,normal,16
3,3,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,incorrect_mask,10
4,4,000001,female,/opt/ml/input/data/train/images/000001_female_...,45,>= 30 and < 60,mask,4
...,...,...,...,...,...,...,...,...
18895,18895,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,normal,12
18896,18896,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,incorrect_mask,6
18897,18897,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,mask,0
18898,18898,006959,male,/opt/ml/input/data/train/images/006959_male_As...,19,< 30,mask,0


In [97]:
class CustomDataset(Dataset):
    def __init__(self, df_data, train=True):
        self.data = df_data
        self.label = self.data['class'].tolist()
        self.img_path = self.data['img_path'].tolist()
        
        self.transform = transforms.ToTensor()

        self.train = train
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index])
        img = img.resize((32, 32))
        img = np.asarray(img)/255
        
        img_tensor = self.transform(img)
        
        if self.train:
            return (self.label[index], img_tensor)
        else:
            return img_tnesor
        
        
        return img

In [98]:
data_test = CustomDataset(train_df)

In [99]:
train_set, valid_set = train_test_split(train_df, test_size=0.20, random_state=77)

In [100]:
print(train_set.shape, valid_set.shape)

(15120, 8) (3780, 8)


In [101]:
custom_dataset = CustomDataset(train_set)

In [102]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(3136, 18)
        
    def forward(self, x):
        x = self.cnn1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [103]:
test_model = CustomModel()

In [104]:
test_model.to(device)

CustomModel(
  (cnn1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=18, bias=True)
)

In [105]:
print(test_model)

CustomModel(
  (cnn1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=18, bias=True)
)


In [106]:
from torch.autograd import Variable

my_dataset_loader =  torch.utils.data.DataLoader(dataset=custom_dataset, batch_size=14, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(test_model.parameters(), lr=1e-2)
# optimizer = optim.Adam(test_model.parameters(), lr=1e-2)

for epoch in range(2):
    for i, (labels, images) in enumerate(my_dataset_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        optimizer.zero_grad()
        
        outputs = test_model(images.float())
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        if i%200 == 0:
            print(f"Eopch: {epoch} | Batch ID: {i} | Loss: {loss.data}")

print("done!")

Eopch: 0 | Batch ID: 0 | Loss: 2.882166624069214
Eopch: 0 | Batch ID: 200 | Loss: 2.4311845302581787
Eopch: 0 | Batch ID: 400 | Loss: 2.2662477493286133
Eopch: 0 | Batch ID: 600 | Loss: 1.6459068059921265
Eopch: 0 | Batch ID: 800 | Loss: 1.3359240293502808
Eopch: 0 | Batch ID: 1000 | Loss: 1.585040807723999
Eopch: 1 | Batch ID: 0 | Loss: 1.901785969734192
Eopch: 1 | Batch ID: 200 | Loss: 1.746011734008789
Eopch: 1 | Batch ID: 400 | Loss: 1.581292748451233
Eopch: 1 | Batch ID: 600 | Loss: 1.0829755067825317
Eopch: 1 | Batch ID: 800 | Loss: 0.8084301948547363
Eopch: 1 | Batch ID: 1000 | Loss: 1.138932466506958
done!


In [107]:
valid_dataset = CustomDataset(valid_set)

valid_dataset_loader =  torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=14, shuffle=False)

In [108]:
targets = []
all_predictions = []

for labels, images in valid_dataset_loader:
    with torch.no_grad():
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        pred = test_model(images.float())
        pred = pred.argmax(dim=-1)
        
        targets.extend(labels.cpu().numpy())
        
        all_predictions.extend(pred.cpu().numpy())

print(f"Accuracy: {accuracy_score(targets, all_predictions)}")
print(f"F1 Score: {np.mean(f1_score(targets, all_predictions, average=None))}")

Accuracy: 0.6145502645502645
F1 Score: 0.3496435481921103
